# Public Editor Convergence Tool 

In [1]:
import numpy as np
import pandas as pd
import re
import os
from collections import Counter

## Goal: ## 
Our goal of this tool is to find the minimum number of users it takes to reach a consensus for each question. We are treating the consensus (the majority of answers) as the "correct answer". Then we are using a bootstrapping method in order to take simple random samples until we reach the correct answer of that specific question. 

## Reading In The Data and Organizing It ##
We'll need to read in our data file and the schema for it. As of right now, we are handling it file by file since each DataHuntAnswers.csv and Scheme.csv are different questions and answers. In the future, we hope we can build a tool to be able to read in all the data easily.

In [2]:
#These are all the tasks we have for now. Requires changes if we have more tasks or the representation of tasks in the 
#datafile names changes.
tasks = ['Evidence', 'Language', 'Probability', 'Reasoning'] 
# tasks = ['Evidence']
data_dir = '../testing-format/' #tailing / in necessary
data_files = os.listdir(data_dir)
data_files
dfs = {}

for t in tasks:
    data_files_t = [path for path in data_files if re.match('.*{}.*'.format(t), path) ]
    dfs[t] = [pd.read_csv(data_dir + data_file_t) for data_file_t in data_files_t]
# df_master = pd.read_csv('../newDataFormat/BETA_Language-2020-01-18T0225-DataHuntSubmitted.csv')
# Functions about schema is depreciated
# schema = pd.read_csv("../urap/Demo1Evi-2019-06-22T0023-Schema.csv")
# backup_schema = schema
# df_master.head()

We will be using the Schema file in order to help us disregard open-ended questions since those types of questions will not have a convergence or a majority answer.

In [3]:
# Depreciated, waiting to be fixed
# textQ = list(schema[schema['question_type'] == 'TEXT']["question_label"])
# textQ

The following code takes in the data and organizes it based on contributor ID. It shows which questions each user had reached as well as their answers to those respective questions.

In [4]:
'''
s: a series

Return:
    A dataframe, hot-encoded in a managable manner
'''
def hotcode_multiple_choices(s):
    keys = set()
    for elem in s:
        if type(elem) == list:
            for key in elem:
                keys.add(key)
    
    encode_data = np.zeros((len(s), len(keys)))
    i = 0
    for elem in s:
        if type(elem) == list:
            j = 0
            for key in keys:
                if key in elem:
                    encode_data[i][j] = 1
                j += 1
        i += 1
    encode_data = pd.DataFrame(encode_data, dtype=np.uint8)
    encode_data.columns = keys
    encode_data.index = s.index
    return encode_data

'''
This function get an article from 2020 feeds and convert it into the format of 2019 feeds so the code below can be reused

df: the dataframe of 2020 format, e.g. BETA_Language-2020-01-18T0225-DataHuntSubmitted.csv
article_id: article_number in df

Return:
    A pandas dataframe similiar to the format of 2019 feeds
'''
def getArticle(df, article_id):
    article = df[df['article_number'] == article_id]
    #We use quiz_taskrun_uuid as dummy index, since it is a primary index for 2019 feed
    tbl = pd.pivot_table(article, values='answer_label', index='quiz_taskrun_uuid', columns='question_label', aggfunc=list)
    for col in tbl.columns:
        helper = tbl[col].apply(lambda x: len(x) if type(x) == list else 0)
        if max(helper) <= 1: #just unwrap from list if everyone only choose one answer
            tbl[col] = tbl[col].apply(lambda x: x[0] if type(x) == list else 0)
        else:
            encoded = hotcode_multiple_choices(tbl[col])
            for col_ in encoded.columns:
                tbl[col_] = encoded[col_]
#             tbl = pd.concat([tbl, encoded], axis=0, join='inner', sort=False)
            tbl = tbl.drop(labels=col, axis=1)
    return tbl

## Bootstrapping ##
We used a Bootstrapping strategy in order to find the number it takes to match the consensus.

We chose a p-value of 0.01 where we want 99% of the time the majority reaches the "correct" consensus. 

In the future, we hope to integrate the user reputation tool and be able to put weights on the users. As of right now, we do not have the tool so the user weights/reputation scores are all just 1. 

The following function is to find the consensus of each question by finding the max of the answers. 

In [5]:
def getConsensus(answers):
    qcounts = Counter(answers)
    maxKey = max(qcounts.keys(), key=lambda key: qcounts[key])
    return maxKey

To help improve the effiency of our code, we converted all of the answers from strings to integers.

In [6]:
def strToInt(lst):
    vals = lst.unique()
    mapper = {vals[i]:i for i in range(len(vals))}
    return lst.replace(mapper)

The following function is to bootstrap and find the minimum number of users needed to find a convergence for each question. If we are unable to reach a confidence of 99% after sampling a max number of users, then we cannot find a convergence. The number of users that we sample can be changed in the future. 

In [7]:
# n = number of answer choices for questions
# c = consensus of entire dataset for question
# answers = answer column

def getN(questionName):
    answers = df[questionName].dropna()
    answers = strToInt(answers)
    n = len(pd.unique(answers))
    c = getConsensus(answers)
    
    for i in range(n + 1, max_group_size):
        count = 0
        for s in range(0, 1000):
            sample = np.random.choice(answers, i, replace=True)
            consensus = getConsensus(sample)
            if consensus == c:
                count += 1
        
        if count/1000 > .99:
            return i
    
#     return "Not converged"
    return -1 #Use -1 to represent not converged data, we are probably just ignoring it

We ran the function using a sample size of 100. We used a print statement in order to figure out the efficiency of it and organized the final results in a panda dataframe.

In [8]:
need_group_for_tasks = {}
for t in tasks:
    needed_groups = []
    for df_master in dfs[t]:
        needed_group = 0

        for article_id in df_master['article_number'].unique():
            df = getArticle(df_master, article_id)
            cols = ['Question', 'Min']
            lst=[]
            max_group_size = 100 # FEEL FREE TO CHANGE THIS

            for q in df.columns[1:]:
                if len(df[q].dropna().unique()) < 20:
                    n = getN(q)
                    lst.append(n)
                    print('Article Number:', article_id, "Question:", q, "Converge:", n)
            max_needed = max(lst)
            if max_needed > needed_group:
                needed_group = max_needed
        needed_groups.append(needed_group)
    need_group_for_tasks[t] = max(needed_groups)

Article Number: 1712 Question: T1.Q11 Converge: -1
Article Number: 1712 Question: T1.Q13 Converge: -1
Article Number: 1712 Question: T1.Q14 Converge: -1
Article Number: 1712 Question: T1.Q3 Converge: 3
Article Number: 1712 Question: T1.Q4 Converge: 68
Article Number: 1712 Question: T1.Q5 Converge: 15
Article Number: 1712 Question: T1.Q6 Converge: 6
Article Number: 1712 Question: T1.Q7 Converge: 2
Article Number: 1712 Question: T1.Q8 Converge: -1
Article Number: 1712 Question: T1.Q1.A1 Converge: 25
Article Number: 1712 Question: T1.Q1.A3 Converge: 3
Article Number: 1712 Question: T1.Q1.A2 Converge: -1
Article Number: 1712 Question: T1.Q12.A2 Converge: 27
Article Number: 1712 Question: T1.Q12.A4 Converge: 21
Article Number: 1712 Question: T1.Q12.A3 Converge: 46
Article Number: 1712 Question: T1.Q12.A1 Converge: 9
Article Number: 1712 Question: T1.Q2.A7 Converge: 3
Article Number: 1712 Question: T1.Q2.A1 Converge: 52
Article Number: 1712 Question: T1.Q2.A4 Converge: 13
Article Number: 171

Article Number: 100002 Question: T1.Q1.A9 Converge: 7
Article Number: 100002 Question: T1.Q1.A10 Converge: 7
Article Number: 100002 Question: T1.Q1.A5 Converge: 3
Article Number: 100002 Question: T1.Q1.A4 Converge: -1
Article Number: 100002 Question: T1.Q1.A12 Converge: 3
Article Number: 100002 Question: T1.Q1.A2 Converge: 27
Article Number: 100002 Question: T1.Q1.A6 Converge: -1
Article Number: 100002 Question: T1.Q1.A1 Converge: 10
Article Number: 100002 Question: T1.Q6.A2 Converge: 13
Article Number: 100002 Question: T1.Q6.A7 Converge: 3
Article Number: 100002 Question: T1.Q6.A8 Converge: 5
Article Number: 100002 Question: T1.Q6.A1 Converge: 7
Article Number: 100002 Question: T1.Q6.A3 Converge: 10
Article Number: 100002 Question: T1.Q6.A4 Converge: 3
Article Number: 100002 Question: T1.Q6.A5 Converge: 5
Article Number: 100003 Question: T1.Q11 Converge: 5
Article Number: 100003 Question: T1.Q12 Converge: -1
Article Number: 100003 Question: T1.Q13 Converge: 5
Article Number: 100003 Qu

Article Number: 100026 Question: T1.Q2 Converge: 2
Article Number: 100026 Question: T1.Q5 Converge: 2
Article Number: 100026 Question: T1.Q6 Converge: 2
Article Number: 1712 Question: T1.Q10 Converge: 2
Article Number: 1712 Question: T1.Q3 Converge: 2
Article Number: 1712 Question: T1.Q9 Converge: 2


In [9]:
for key in need_group_for_tasks.keys():
    need_group_for_tasks[key] = [need_group_for_tasks[key]]

In [10]:
pd.DataFrame.from_dict(need_group_for_tasks).to_csv('needed_people_for_tasks.csv')